# Tests avec Stanza
Voir la documentation : https://stanfordnlp.github.io/stanza/

In [12]:
import re

from bs4 import BeautifulSoup
import stanza

## Installation de la pipeline

Il est nécessaire d'installer la chaîne de traitement en langue française pour traiter du texte avec Stanza. 

In [4]:
# Pour télécharger la chaîne de traitement de Stanza pour la langue française
stanza.download('fr')

2021-08-06 22:15:25 INFO: Downloading default packages for language: fr (French)...
2021-08-06 22:15:27 INFO: File exists: /home/hugoscheithauer/stanza_resources/fr/default.zip.
2021-08-06 22:15:31 INFO: Finished downloading models and saved to /home/hugoscheithauer/stanza_resources.


## Pré-traitement du XML PAGE

On récupère le XML PAGE, on le parse, et on essaye de reconstituer chaque enregistrement grâce à l'annotation sémantique réalisée dans eScriptorium.

Le type:first_line correspond à chaque première ligne d'enregistrement. La première ligne du registre, indiquant la date, possède également ce label.

Le modèle de segmentation qui a été utilisée attribue le label `column_odd` ou `column_pair`. La colonne centrale pourrait être récupéré en cherchant à extraire la troisième colonne pair. 

A chaque fin d'enregistrement, on ajoute un "@ " pour indiquer la fin de l'unité sémantique. De la sorte, on reconstitue artificiellement la structure de la page.

On segmente ensuite les enregistrements en dehors de la pipeline de NLP Stanza, car je n'ai pas trouvé de sentencizer dédié pour le moment. On reproduit l'action d'un sentencizer en faisant un .split() sur "@".

In [5]:
tag_list = []

# On ouvre le fichier XML PAGE
with open("../../corpus_test/lectaurep/doc_28_sample/annotated_first_lines/FRAN_0025_0029_L-0.xml") as file:
    parsed_sample = BeautifulSoup(file, "lxml-xml")


# On effectue les transformations pour segmenter artificiellement les enregistrements
textline_tags = parsed_sample.find_all("TextLine")
for i in range(len(textline_tags)):
    if textline_tags[i].has_attr('custom'):
        if textline_tags[i]['custom'] == 'structure {type:first_line;}':
                # On modifie l'index précédent quand on arrive sur une balise avec @custome == 'structure {type:first_line;}'
                unicode = textline_tags[i-1].find('Unicode')
                unicode.append("@")
      
    
# On récupère les tags Unicode modifiés
unicode_tags = parsed_sample.find_all("Unicode")
text_tags_list = [tag.text for tag in unicode_tags]
# On join() les index de la liste
text = ''.join(text_tags_list)
# On essaye de reconstituer les noms aglutinés à l'aide des majuscules
text = re.sub(r"(\w)([A-Z])", r"\1 \2", text)
# On segmente les enregistrements à la main avec .split()
text_sent = text.split("@")
# On obtient une liste où chaque index correspond à un enregistrement

# On supprime le dernier index qui est vide (''), créé à cause du split sur @
text_sent = text_sent[:-1]
text_sent

['An 1901, mois de Étévrier',
 'Dupuis (par Pierre) à Paris, rue Turot 4, à Catherine Bigol, safen',
 'a Oo (par Mve) s.n. à',
 'Muret (de Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine Lemaire, dt à Paris, même adreré (Sléparation de biens',
 "Duboscqfet titre de 380+. de rente 3% au nom de: Debuisson, Juatine Adèle Se de Pnles) et autres, à Paris, rue del'Orcade 21",
 'Rousfpa Pdmord Harcisse Bt a Paris, rue Canaletti 15, et autrespr touchor V',
 'Chabr (par Marie Louise Jatoune, Ve de Marie Eusébe Maseine Bt à Billancourt ronte de Sersailles. 18, à Ougène Vahrin et Marceline Fhalino, dt à Billancourt, rue Mationale 31, de 21303.88',
 'Gauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère',
 'Oussonppar Geordes Ernert Léou dt à Paris, rue Lamarch 144, à sonpère',
 "Lecrosnier (etlivret de Caistte d'Epargne de Paris, de157.19, aunonde Pierre Pugute) décédé en sondomicilé à P Paris, rue Descartes 21, le 16 8b

## Traitements NLP

In [6]:
# On définit la chaîne de traitement qui nous intéresse, ici le tokenizer et le ner
nlp = stanza.Pipeline(lang='fr', processors='tokenize, ner')

2021-08-06 22:15:31 WARNING: Language fr package default expects mwt, which has been added
2021-08-06 22:15:31 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| ner       | wikiner |

2021-08-06 22:15:31 INFO: Use device: cpu
2021-08-06 22:15:31 INFO: Loading: tokenize
2021-08-06 22:15:31 INFO: Loading: mwt
2021-08-06 22:15:31 INFO: Loading: ner
2021-08-06 22:15:32 INFO: Done loading processors!


On utilise une comprehensive list pour appliquer la pipeline Stanza sur chaque index de notre liste text_sent

In [7]:
%%time
doc = [nlp(i) for i in text_sent]

CPU times: user 16.8 s, sys: 65.6 ms, total: 16.8 s
Wall time: 11.9 s


In [8]:
%%time
# Pour comparer le temps que cela met seulement sur une chaîne de caractères et non une liste
doc2 = nlp(text)

CPU times: user 22.9 s, sys: 320 ms, total: 23.2 s
Wall time: 8.39 s


## NER

On affiche, pour chaque enregistrement, les entités nommées reconnues par la pipeline Stanza.

In [10]:
for sent in doc:
    print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in sent.ents], sep='\n')
    print("\n")

entity: Étévrier	type: LOC


entity: Dupuis	type: ORG
entity: Pierre	type: MISC
entity: Paris	type: LOC
entity: rue Turot	type: LOC
entity: Catherine Bigol	type: PER


entity: Oo	type: LOC
entity: Mve	type: ORG


entity: Muret	type: PER
entity: Paul Louis Georges) dt à Paris	type: PER
entity: Bd Rochechouart	type: PER
entity: Marie Joséphine Lemaire	type: PER
entity: Paris	type: LOC


entity: Duboscqfet	type: LOC
entity: Debuisson	type: LOC
entity: Juatine Adèle Se de Pnles	type: PER
entity: Paris	type: LOC
entity: rue del'Orcade	type: LOC


entity: Rousfpa Pdmord Harcisse Bt	type: LOC
entity: Paris	type: LOC
entity: rue Canaletti	type: LOC
entity: V	type: ORG


entity: Chabr	type: PER
entity: Marie Louise Jatoune	type: PER
entity: Ve de Marie Eusébe Maseine Bt à Billancourt	type: PER
entity: Sersailles	type: LOC
entity: Ougène Vahrin	type: PER
entity: Marceline Fhalino	type: PER
entity: Billancourt	type: LOC
entity: rue Mationale	type: LOC


entity: Marie Cexandinte	type: PER


On affiche, pour chaque enregistrement, les token reconnus par la pipeline Stanza.

In [11]:
for idx in doc:
    print(*[f'token: {token.text}\tner: {token.ner}' for sent in idx.sentences for token in sent.tokens], sep='\n')

token: An	ner: O
token: 1901	ner: O
token: ,	ner: O
token: mois	ner: O
token: de	ner: O
token: Étévrier	ner: S-LOC
token: Dupuis	ner: S-ORG
token: (	ner: O
token: par	ner: O
token: Pierre	ner: S-MISC
token: )	ner: O
token: à	ner: O
token: Paris	ner: S-LOC
token: ,	ner: O
token: rue	ner: B-LOC
token: Turot	ner: E-LOC
token: 4	ner: O
token: ,	ner: O
token: à	ner: O
token: Catherine	ner: B-PER
token: Bigol	ner: E-PER
token: ,	ner: O
token: safen	ner: O
token: a	ner: O
token: Oo	ner: S-LOC
token: (	ner: O
token: par	ner: O
token: Mve	ner: S-ORG
token: )	ner: O
token: s.n.	ner: O
token: à	ner: O
token: Muret	ner: S-PER
token: (	ner: O
token: de	ner: O
token: Paul	ner: B-PER
token: Louis	ner: I-PER
token: Georges	ner: I-PER
token: )	ner: I-PER
token: dt	ner: I-PER
token: à	ner: I-PER
token: Paris	ner: E-PER
token: ,	ner: O
token: Bd	ner: B-PER
token: Rochechouart	ner: E-PER
token: 68	ner: O
token: ,	ner: O
token: et	ner: O
token: Marie	ner: B-PER
token: Joséphine	ner: I-PER
token: Lem